# Initialization and Set Up


In [1]:
!pip install google-generativeai
!pip install pymupdf

print("Install done")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Install done


## Test Gemini API Key
- gemini-2.0-flash

In [2]:
import google.generativeai as genai

/Users/mannanxanand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [4]:
api_key = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=api_key)

# Initialize the Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")

In [5]:
# Ask a question
response = model.generate_content("Explain quantum computing in simple terms.")

# Print the response
print(response.text)


Imagine a light switch. A regular computer bit is like that switch: it can be either ON (1) or OFF (0).  It can only be in one of those states at a time.

Quantum computers use **qubits**. Think of a **dimmer switch** instead of a light switch.  It can be ON, OFF, *or somewhere in between*.  This "in between" state is called **superposition**.  It's like the qubit is a 0 and a 1 *at the same time*.

Another important concept is **entanglement**. Imagine two of our dimmer switches are linked together in a strange way.  When you change the setting of one, the other instantly changes too, even if they're far apart!  Entanglement allows qubits to work together in a powerful way.

**So, what does this all mean?**

Because qubits can be in multiple states at once (superposition) and linked together (entanglement), quantum computers can explore many possibilities simultaneously.  Think of it like trying to find your way through a maze.

*   **Regular computer:** Tries one path at a time. If i

In [6]:
import fitz
import difflib
import re
import json

### Error log

In [7]:
# Let it be global
error_log = """Error log\n"""

# Starting of pipeline 
1. reads legal documents
2. calls LLM to add perturbations
3. creates output in json format
4. stores output files in benchmark dataset

## Change source folder and destination folder


In [8]:
# Edit these as needed

# Folder paths
"""
folder_path_read = folder path to read the pdfs, put the root folder here and it picks out all pdfs

folder_path_json = folder path to save the perturbation json files

folder_path_save = folder path to save the modified perturbed text files
"""

folder_path_read = "full_contract_pdf/Part_I/"

folder_path_json = "benchmark_dataset_v1/inconsistent_inText_json/"

folder_path_save = "benchmark_dataset_v1/inconsistent_inText_contradiction/"

# Error log file name
error_log_name = "error_log_misaligned_terminology_inText.txt"

# Change this as needed
#start_folder = ""
#start_folder = "full_contract_pdf/Part_I/License_Agreements/"

# Index of subfolder to start with from the root folder in folder_path_read
start_index = 0

"""Prompts"""
# See function generate_perturbation_new

'Prompts'

Mannan's Recursive Function Cause My Stuff Always Errors:

In [9]:
def read_legal_files_recursive(root_folder):
    """
    Recursively reads all PDF files under the given root folder and 
    returns a dictionary with keys as the full file paths and values as the file content.
    """
    legal_documents = {}
    for dirpath, _, filenames in os.walk(root_folder):
        for file_name in filenames:
            if file_name.endswith(".pdf"):
                file_path = os.path.join(dirpath, file_name)
                legal_documents[file_path] = read_pdf(file_path)
            else:
                print(f"Skipping unsupported file: {file_name}")
    return legal_documents


## Retrieve content for each legal document


In [10]:
def get_end_folders(root_folder, skip_folder=".ipynb_checkpoints"):
    end_folders = []

    for dirpath, dirnames, _ in os.walk(root_folder, topdown=True):
        # Remove the folders that should be skipped
        dirnames[:] = [d for d in dirnames if d != skip_folder]

        # If there are no subdirectories left, it's an end folder
        if not dirnames:
            end_folders.append(os.path.join(dirpath, ""))  # Ensure trailing backslash

    return end_folders

# Example usage:
# result = get_end_folders("full_contract_pdf")
# print(result)

In [11]:
def read_pdf(file_path):
    """Reads a PDF file"""
    doc = fitz.open(file_path)
    return "\n".join([page.get_text() for page in doc])

def read_legal_files(folder_path):
    """Reads all legal files in the folder and returns a dictionary with file names and content."""
    legal_documents = {}

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            legal_documents[file_name] = read_pdf(file_path)
        else:
            print(f"Skipping unsupported file: {file_name}")

    return legal_documents

In [12]:
# Read legal files
legal_docs = read_legal_files(folder_path_read)

# Display first document
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---\n{content[:500]}...\n")
    break

Skipping unsupported file: Promotion
Skipping unsupported file: Non_Compete_Non_Solicit
Skipping unsupported file: Joint Venture
Skipping unsupported file: Transportation
Skipping unsupported file: Endorsement
Skipping unsupported file: Affiliate_Agreements
Skipping unsupported file: .DS_Store
Skipping unsupported file: Development
Skipping unsupported file: Co_Branding
Skipping unsupported file: Strategic Alliance
Skipping unsupported file: Franchise
Skipping unsupported file: License_Agreements
Skipping unsupported file: Manufacturing
Skipping unsupported file: Reseller
Skipping unsupported file: Supply
Skipping unsupported file: Sponsorship
Skipping unsupported file: Distributor
Skipping unsupported file: Hosting
Skipping unsupported file: Marketing
Skipping unsupported file: Outsourcing
Skipping unsupported file: Maintenance
Skipping unsupported file: Service
Skipping unsupported file: IP


In [13]:
# Printing all file names that were accepted
print(f"Total files from {folder_path_read}: {len(legal_docs)}") 
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---...\n")

Total files from full_contract_pdf/Part_I/: 0


## Prompt to read through legal file and insert different types of perturbations
- 10 different types of prompts to switch
- returns output in json format, which would be considered as lock-file

In [14]:
def generate_perturbation_new(original_text, file_name, prompt):    
    """Generates a perturbed version of the legal document section in structured JSON format."""

    prompt = f"""You are a financial contract auditor ensuring consistency in payment terms. Your task is to introduce a conflict between different payment deadline clauses.

    ### **Definition:**
    Inconsistencies also lead to **in-text contradictions** when **different sections of a contract provide conflicting deadlines, obligations, or penalties**. This creates ambiguity regarding which terms should be enforced, leading to disputes over contractual obligations.
    
    ### **Step-by-Step Instructions:**
    1. Identify a **key term** in the contract.
    2. Modify its definition in different sections so that they **conflict**.
    3. Ensure the contradiction creates **uncertainty in enforcement**.
    4. For that perturbation, make sure in the file there should be **2-3** of them. 
    5. Output the modified contract in structured JSON format.
    6. Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.
    
    --------
    
    ### **Examples of Conflicting Payment Terms:**
    
    **Example 1:**
    - **Section 4:** "Invoices must be paid within 45 days."
    - **Section 12:** "Late fees apply if invoices are unpaid after 30 days."
    - **Explanation:** One section allows **45 days**, but the other **charges late fees after 30 days**, creating ambiguity.
    
    **Example 2:**
    - **Section 6:** "Security deposits are fully refundable."
    - **Section 9:** "Security deposits are non-refundable."
    - **Explanation:** It is unclear whether **deposits must be returned**.
    
    **Example 3:**
    - **Section 3:** "Early termination fees apply if the contract is ended before 12 months."
    - **Section 10:** "Clients may terminate at any time without penalty."
    - **Explanation:** Conflicting clauses **cause uncertainty about termination fees**.
    
    **Example 4:**
    - **Section 5:** "Payment plans must be completed within 6 months."
    - **Section 11:** "Customers may extend payment plans up to 12 months."
    - **Explanation:** Different sections **contradict the maximum payment period**.
    
    **Example 5:**
    - **Section 8:** "Interest on late payments accrues at 5% per month."
    - **Section 14:** "No interest will be charged on late payments."
    - **Explanation:** **Conflicting interest policies** could lead to legal disputes.
    
    **Example 6:**
    - **Original:** "For the purposes of this Agreement, the term 'Affiliate' shall refer to any entity that is controlled by, controls, or is under common control with either party. The term 'control' shall mean ownership of at least fifty-one percent (51%) of the voting equity in such entity. Affiliates shall be entitled to the same rights and obligations as the contracting party."
    - **Modified:** "For the purposes of this Agreement, the term 'Affiliate' shall refer to any entity that is affiliated with, works in conjunction with, or has a strategic partnership with either party. Affiliates may be granted certain rights and obligations as determined by the contracting party on a case-by-case basis."
    - **Explanation:** The modified text **creates an in-text contradiction** by **changing the definition of "Affiliate"** to include strategic partnerships and loosely affiliated entities, rather than just entities **under majority control**. This contradicts other contract sections that **rely on the original definition** (51% ownership) to determine rights and obligations. Additionally, **allowing case-by-case determination** undermines the original intent of treating affiliates the same as contracting parties, introducing **legal uncertainty** and **potential disputes over obligations**.
    ---
    
    ### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Inconsistencies - In Text Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}
    
    Below is the original legal text:
    -------------------
    {original_text}
    -------------------
    
    Now, return ONLY the structured JSON object with the modified text and explanation.
    """
    
    global error_log
    response = None
    response_text = None
    try:
        response = model.generate_content(prompt)
        response_text = response.text
    except ValueError as e:
        if "reciting from copyrighted material" in str(e):
            print("Error: The model was reciting from copyrighted material. Please modify your prompt.")
        error_log += f"""\nIn {file_name}: 
        Error name: ValueError
        Error message: {e}\n"""
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        error_log += f"""\nIn {file_name}: 
        Error name: Exception
        Error message: {e}\n"""

    
    return response_text if response else "ERROR: No response from API"

## Applies perturbations to files and stores in json format

In [15]:
def apply_perturbations(folder_path_read, folder_path_json, folder_path_save, prompt):
    """
    Reads all PDF files in a folder, applies structural-legal perturbations,
    writes JSON and modified text outputs, and returns the last perturbed JSON 
    (or None if no PDFs were processed).
    """
    legal_docs = read_legal_files(folder_path_read)
    global error_log
    
    last_perturbed_json = None
    
    for i, (file_name, content) in enumerate(legal_docs.items()):
        print("________________________________________________________________________")
        print(f"Processing {file_name}...")
        
        perturbed_json = generate_perturbation_new(content, file_name, prompt)
        
        # If the generation function returned None or a non-string, or if it’s the special “copyright” case, skip
        if not perturbed_json or not isinstance(perturbed_json, str):
            print("Skipping this file because generation failed or returned a non-string.")
            continue
        
        # If the model complained about copyrighted material, skip
        if "Error: The model was reciting from copyrighted material." in perturbed_json:
            print("Skipping this file due to copyright recitation error.")
            continue
        
        # If we have our “no response from API” placeholder, skip as well
        if perturbed_json == "ERROR: No response from API":
            continue
        
        # Now that we know we have a valid string, do the cleanup
        clean_json_text = re.sub(r"```json|```", "", perturbed_json).strip()
        
        try:
            perturbed_data = json.loads(clean_json_text)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for {file_name}, writing into logs and skipping...")
            error_log += f"""\nIn {file_name}: 
            Error name: JSONDecodeError
            Error message: {e}\n"""
            continue
        
        # Save the JSON
        base_file_name = os.path.basename(file_name)
        json_output_path = os.path.join(folder_path_json, f"perturbed_{base_file_name}.json")
        
        try:
            with open(json_output_path, "w", encoding="utf-8") as f:
                # Put data into a list if your pipeline expects a list of perturbations
                json.dump([perturbed_data], f, indent=4, ensure_ascii=False)
        except FileNotFoundError as e:
            print(f"An error occurred while writing to the file: {e}")
            error_log += f"""\nIn {file_name}: 
            Error name: FileNotFoundError
            Error message: {e}\n"""
            continue
        except IOError as e:
            print(f"An error occurred while writing to the file: {e}")
            error_log += f"""\nIn {file_name}: 
            Error name: IOError
            Error message: {e}\n"""
            continue
        
        # Apply the perturbations from JSON
        modified_contract = apply_perturbation_from_json(content, json_output_path, folder_path_save)
        print(f"All perturbations saved in {folder_path_save}")
        
        # Keep track if you want the last JSON
        last_perturbed_json = perturbed_json
    
    return last_perturbed_json


In [16]:
# def apply_perturbations(folder_path_read, folder_path_json, folder_path_save, prompt):
#     legal_docs = read_legal_files(folder_path_read)

#     global error_log

#     for i, (file_name, content) in enumerate(legal_docs.items()):
#         print("________________________________________________________________________")
#         results = []
#         # if i >= 10:  # Stop after processing 5 documents
#         #      break
#         print(f"Processing {file_name}...")
#         #perturbed_json = generate_perturbation(content, file_name, perturbation_type)
#         perturbed_json = generate_perturbation_new(content, file_name, prompt)

#         # If there is no returned json, return this message
#         if perturbed_json.__eq__("ERROR: No response from API"):
#             continue
        
#         #print("This is the perturbed json:", perturbed_json)
#         clean_json_text = re.sub(r"```json|```", "", perturbed_json).strip()

#         # print('this is json:', clean_json_text)
#         try:
#             # Convert response into a Python dictionary
#             perturbed_data = json.loads(clean_json_text)
#             results.append(perturbed_data)
#         except json.JSONDecodeError as e:
#             print(f"Error parsing JSON for {file_name}, writing into logs and skipping...")
#             error_log += f"""\nIn {file_name}: 
#             Error name: JSONDecodeError
#             Error message: {e}\n"""
#             continue
            
#         # Save the JSON output using only the base file name
#         base_file_name = os.path.basename(file_name)  # Extract just the file name without the full path
#         json_output_path = os.path.join(folder_path_json, f"perturbed_{base_file_name}.json")
#         json_output_path = json_output_path.strip()

#         try:
#             with open(json_output_path, "w", encoding="utf-8") as f:
#                 json.dump(results, f, indent=4, ensure_ascii=False)
#         except FileNotFoundError as e:
#             print(f"An error occurred while writing to the file: {e}")
#             error_log += f"""\nIn {file_name}: 
#         Error name: FileNotFoundError
#         Error message: {e}\n"""
#             continue
#         except IOError as e:
#             print(f"An error occurred while writing to the file: {e}")
#             error_log += f"""\nIn {file_name}: 
#         Error name: IOError
#         Error message: {e}\n"""
#             continue
#         except json.JSONEncodeError as e:
#             print(f"An error occurred while encoding JSON: {e}")
#             error_log += f"""\nIn {file_name}: 
#         Error name: JSONEncodeError
#         Error message: {e}\n"""
#             continue

#         # Apply the perturbation from JSON to text
#         modified_contract = apply_perturbation_from_json(content, json_output_path, folder_path_save)

#         print(f"All perturbations saved in {folder_path_save}")

#         return perturbed_json


In [17]:
def normalize_text(text):
    """
    Normalizes text by removing extra spaces, line breaks, and ensuring consistent spacing. Helper function.
    """
    text = text.replace("\n", " ")  # Replace newlines with space
    text = re.sub(r"\s+", " ", text).strip()  # Replace multiple spaces with a single space
    return text

## Create and store tagged modified file from its respective json log file

In [18]:
def apply_perturbation_from_json(original_text, json_file, output_folder="test_benchmark_dataset/"):
    """
    Reads the JSON metadata and applies the described perturbations to the original document,
    adding unique <*$p$*> markers around the modified sections.

    Parameters:
    - original_text (str): The original contract text.
    - json_file (str): Path to the JSON file containing the perturbation details.
    - output_folder (str): Folder to save the modified contract.

    Returns:
    - modified_text (str): The full modified document.
    """

    global error_log  # so we can append to error_log

    # If the JSON file itself doesn't exist, skip
    if not os.path.exists(json_file):
        print(f"File '{json_file}' does not exist. Skipping execution.")
        return None

    print("json file:", json_file)

    # Load the JSON metadata
    with open(json_file, "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # If the JSON is a list, take the first element as the actual data
    if isinstance(json_data, list) and len(json_data) > 0:
        json_data = json_data[0]

    # Extract the file name from the JSON
    # (In case "file_name" is missing for some reason, we provide a fallback.)
    current_file_name = json_data.get("file_name", "UNKNOWN_FILE")

    # Normalize the original contract text
    normalized_text = normalize_text(original_text)
    if not normalized_text:
        print("Warning: The original text is empty. Skipping file.")
        return "EMPTY CONTENT"

    # Prepare to hold the final text after all perturbations
    modified_text = normalized_text

    # Loop over each perturbation and apply it
    for perturbation in json_data["perturbation"]:
        original_section = normalize_text(perturbation["original_text"])
        changed_section = normalize_text(perturbation["changed_text"])

        # Wrap the changed section in marker tags
        marked_section = f"<*$p$*>{changed_section}<*$p$*>"

        # Attempt to replace in the main text
        if original_section in modified_text:
            modified_text = modified_text.replace(original_section, marked_section)
        else:
            # If we can't find the original section, log it and stop
            e = f"Could not find section in text: {original_section}"
            print("Warning:", e)
            error_log += f"""\nIn {current_file_name}: 
Error name: FileModifyError
Error message: {e}\n"""
            return "COULD NOT MODIFY FILE"

    print("File modified, saving...")

    # Construct the output text file name using the JSON's file_name
    modified_file_name = f"modified_{current_file_name}.txt"
    modified_file_path = os.path.join(output_folder, modified_file_name)

    # Write out the modified text
    with open(modified_file_path, "w", encoding="utf-8") as file:
        file.write(modified_text)

    print(f"File '{json_file}' loaded and written.")
    return modified_text


In [19]:
# def apply_perturbation_from_json(original_text, json_file, output_folder="test_benchmark_dataset/"):
#     """
#     Reads the JSON metadata and applies the described perturbations to the original document,
#     adding unique <*$p$*> markers around the modified sections.

#     Parameters:
#     - original_text (str): The original contract text.
#     - json_file (str): Path to the JSON file containing the perturbation details.
#     - output_folder (str): Folder to save the modified contract.

#     Returns:
#     - modified_text (str): The full modified document.
#     """

#     global error_log
#     # Ensure the output directory exists
#     #os.makedirs(output_folder, exist_ok=True)

#     if not os.path.exists(json_file):  # Check if the file does NOT exist
#         print(f"File '{json_file}' does not exist. Skipping execution.")
#         return null
    
#     print("json file:", json_file)
    
#     # Load the JSON metadata
#     with open(json_file, "r", encoding="utf-8") as file:
#         json_data = json.load(file)

#     #print("File successfully loaded") 
#     if isinstance(json_data, list) and len(json_data) > 0:
#         json_data = json_data[0]  # Extract the first item in the list
    
#     # Normalize the original contract text
#     normalized_text = normalize_text(original_text)
#     normalized_text = normalize_text(original_text)
#     if not normalized_text:
#         print("Warning: The original text is empty. Skipping file.")
#         return "EMPTY CONTENT"


#     # Apply modifications with unique markers
#     modified_text = normalized_text
    
#     for perturbation in json_data["perturbation"]:
#         # Normalize both original and the changed section of text
#         original_section = normalize_text(perturbation["original_text"])  
#         #print("this is original text:", original_section)
#         changed_section = normalize_text(perturbation["changed_text"])
#         #print("this is the changed text:", changed_section)
        
#         # Wrap changed section with unique <*$p$*> markers
#         marked_section = f"<*$p$*>{changed_section}<*$p$*>"

#         # Replace original section with marked modified section
#         if original_section in modified_text:
#             modified_text = modified_text.replace(original_section, marked_section)
#         else:
#             e = f"Could not find section in text: {original_section}"
#             print("Warning: " + e)
#             error_log += f"""\nIn {file_name}: 
#             Error name: FileModifyError
#             Error message: {e}\n"""
#             error = "COULD NOT MODIFY FILE"
#             return error

#     print("File modified, saving...")
#     # Save the modified contract as a new file
#     modified_file_name = f"modified_{json_data['file_name']}.txt"
#     modified_file_path = os.path.join(output_folder, modified_file_name)

#     with open(modified_file_path, "w", encoding="utf-8") as file:
#         file.write(modified_text)

#     print(f"File '{json_file}' loaded and written.") 
#     return modified_text

## Functions to clean and apply highlighting to the perturbed legal documents

In [20]:
def highlight_changes(original, modified):
    """Compares original and modified text and marks changes."""
    original_lines = original.split("\n")
    modified_lines = modified.split("\n")

    diff = difflib.ndiff(original_lines, modified_lines)
    highlighted = []
    
    for line in diff:
        if line.startswith("+ "):  # Added text
            highlighted.append(f"[MODIFIED] {line[2:]}")
        elif line.startswith("- "):  # Removed text
            highlighted.append(f"[REMOVED] {line[2:]}")
        else:
            highlighted.append(line[2:])  
    
    return "\n".join(highlighted)

In [21]:
def extract_clean_text(perturbed_text):
    """
    Removes [MODIFIED], [REMOVED] tags and explanations, leaving only the modified version.
    """
    # Remove any [MODIFIED] or [REMOVED] markers
    clean_text = re.sub(r"\[MODIFIED\]|\[REMOVED\]", "", perturbed_text)
    
    # Remove explanations (assuming they are after a certain marker like "Explanation:")
    clean_text = re.sub(r"Explanation:.*", "", clean_text, flags=re.DOTALL)
    
    # Clean up extra spaces that may remain after removal
    clean_text = re.sub(r"\n\s*\n", "\n", clean_text).strip()
    
    
    
    return clean_text

In [22]:
# Destination directory creation and check
os.makedirs(folder_path_json, exist_ok=True)
os.makedirs(folder_path_save, exist_ok=True)

# Get all end folders, make it quick
end_folder_names = get_end_folders(folder_path_read)
# perturbation_type = "contradiction"  # Change to "ambiguity", "omission", etc.
# perturbed_legal_docs = apply_perturbations(folder_path_read, folder_path_json, folder_path_save, perturbation_type, prompt)

# Find the index of the start folder
# if start_folder in end_folder_names:
#     start_index = end_folder_names.index(start_folder)
# else:
#     start_index = 0  # Default to starting from the beginning if folder not found

# Initialize perturbed_json outside the loop to avoid UnboundLocalError
perturbed_json = "No perturbations applied"

for folder_name in end_folder_names[start_index:]:
    print("\nCurrently in " + folder_name + "\n")
    perturbed_legal_docs = apply_perturbations(folder_name, folder_path_json, folder_path_save, "")
    if perturbed_legal_docs:
        perturbed_json = perturbed_legal_docs


Currently in full_contract_pdf/Part_I/Promotion/

________________________________________________________________________
Processing DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_11414857_EX-10.2_Promotion Agreement.pdf...
json file: benchmark_dataset_v1/inconsistent_inText_json/perturbed_DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_11414857_EX-10.2_Promotion Agreement.pdf.json
File modified, saving...
File 'benchmark_dataset_v1/inconsistent_inText_json/perturbed_DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_11414857_EX-10.2_Promotion Agreement.pdf.json' loaded and written.
All perturbations saved in benchmark_dataset_v1/inconsistent_inText_contradiction/
________________________________________________________________________
Processing VnueInc_20150914_8-K_EX-10.1_9259571_EX-10.1_Promotion Agreement.pdf...
json file: benchmark_dataset_v1/inconsistent_inText_json/perturbed_VnueInc_20150914_8-K_EX-10.1_9259571_EX-10.1_Promotion Agreement.pdf.json
File modified, saving...
File 'benchmar

# End of Program

In [23]:
# Output error log txt file
# print(error_log)
with open(error_log_name, "w", encoding="utf-8") as file:
    file.write(error_log)

print("Error log written successfully.")

Error log written successfully.


In [24]:
print("EOP")

EOP
